In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely as shp

from glob import glob
from os.path import join
from pandana.loaders import osm

### Construct a node-edge network of all the intersections within our bounding box. This allows us to compute distance calulations and aggregates with point data on this network

In [2]:
net = osm.pdna_network_from_bbox(45.1453, -123.1095, 45.7859, -122.2575, network_type='drive')

Requesting network data within bounding box from Overpass API in 4 request(s)
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](45.14530000,-123.10950000,45.46678913,-122.68170813);>;);out;'}"
Downloaded 15,947.2KB from www.overpass-api.de in 4.84 seconds
Posting to http://www.overpass-api.de/api/interpreter with timeout=180, "{'data': '[out:json][timeout:180];(way["highway"]["highway"!~"cycleway|footway|path|pedestrian|steps|track|proposed|construction|bridleway|abandoned|platform|raceway|service"]["motor_vehicle"!~"no"]["motorcar"!~"no"]["service"!~"parking|parking_aisle|driveway|emergency_access"](45.46639816,-123.10950000,45.78669410,-122.67989024);>;);out;'}"
Downloaded 20,301.9KB fro

In [3]:
# precompute a distance of 3000 meters, we won't be doing calulations further than that
net.precompute(3000)

### Preprocess point of interest data and add to the network

We can 'set' points of interest to the network (e.g. grocery stores) to later perform aggregations on. We do this by simply passing vectors of lats and longs, a label for the points and the max distance we care about in future calulations. Some of the data is in ESRI Shapefile format so they need to be loaded and have their coordinates extracted.

In [4]:
def process_pois(rootdir, network, max_map=250):
    # iterate through point shapefiles
    for data in glob(join('./data/pois', '*')):
        print('Processing {}...'.format(data))
        df = gpd.read_file(data)
        # confirm the projection is WGS84
        if df.crs != {'init': 'epsg:4326'}:
            df = df.to_crs({'init': 'epsg:4326'})
        xs = df.apply(lambda row: row['geometry'].x, axis=1)
        ys = df.apply(lambda row: row['geometry'].y, axis=1)
        
        network.set(network.get_node_ids(xs, ys, max_map), variable=None, name=data.split('/')[3])
    print('Done!')

In [5]:
process_pois('./data/pois', net)

Processing ./data/pois/grocery_store...
Processing ./data/pois/lrt_stop...
Processing ./data/pois/tran_cen...
Done!


#### The OSM data already has lat longs but needs to be separated into groups (i.e. unconcatenated)

In [6]:
# OSM already has lat longs so process these can essentially be added to the network directly
osm = pd.read_csv('./data/osm.csv')

In [7]:
osm.head(2)

,id,lat,lon,name,amenity,alias
0,287206333,45.534615,-122.916619,Swagath,restaurant,food_drink
1,287207155,45.550412,-122.904555,Pho Tango,restaurant,food_drink


In [8]:
osm.alias.unique()

array(['food_drink', 'cafe', 'college', 'entertainment'], dtype=object)

#### Separate out different types of amenities

In [9]:
fd = osm[osm['alias'] == 'food_drink'][['id', 'lon', 'lat']]
fd.index.name = 'fd'

In [10]:
cafe = osm[osm['alias'] == 'cafe'][['id', 'lon', 'lat']]
cafe.index.name = 'cafe'

In [11]:
college = osm[osm['alias'] == 'college'][['id', 'lon', 'lat']]
college.index.name = 'college'

In [12]:
ent = osm[osm['alias'] == 'entertainment'][['id', 'lon', 'lat']]
ent.index.name = 'ent'

In [13]:
# set points to the net
for df in [fd, cafe, college, ent]:
    net.set(net.get_node_ids(df['lon'], df['lat'], 250), variable=None, name=df.index.name)

### Load the data we actually care about (observations aka properties) and assign each to a node

In [14]:
props = gpd.read_file('./data/properties')

In [15]:
props.crs

{'init': 'epsg:4326'}

In [16]:
xs = props.apply(lambda row: row['geometry'].x, axis=1)

In [17]:
ys = props.apply(lambda row: row['geometry'].y, axis=1)

In [18]:
props['node_id'] = net.get_node_ids(xs, ys)

In [19]:
props.head()

,geometry,propertyid,node_id
0,POINT (-122.535763 45.781812),8021379,47251805
1,POINT (-122.5075064 45.6203498),9096130,47307668
2,POINT (-122.513618 45.622437),7769079,47330355
3,POINT (-122.669628 45.7208891),6058580,3686142098
4,POINT (-122.3796812 45.589126),5998841,47211778


### Perform aggregates
We now have the following ammeneties set on the net:
* grocery_store
* lrt_stop
* tran_cen
* fd
* cafe
* college
* ent

This is nice because we can quickly and easily change the type of aggregate and the radius of the search

In [20]:
grocery = net.aggregate(2000, type='count', decay='flat', name='grocery_store')
grocery.name = 'grocery_2k'
light_rail = net.aggregate(2000, type='count', decay='flat', name='lrt_stop')
light_rail.name = 'lightrail_2k'
transit_centers = net.aggregate(2000, type='count', decay='flat', name='tran_cen')
transit_centers.name = 'transit_cen_2k'
food_drink = net.aggregate(2000, type='count', decay='flat', name='fd')
food_drink.name = 'food_drink_2k'
caf = net.aggregate(2000, type='count', decay='flat', name='cafe')
caf.name = 'cafe_2k'
coll = net.aggregate(2000, type='count', decay='flat', name='college')
coll.name = 'college_2k'
entertain = net.aggregate(2000, type='count', decay='flat', name='ent')
entertain.name = 'entertainment_2k'

### Join the aggregates to our observations

In [21]:
props.shape

(409, 3)

In [22]:
sub = props
for agg in [grocery, light_rail, transit_centers, food_drink, caf, coll, entertain]:
    sub = sub.join(agg, on='node_id', how='inner')

In [23]:
sub.head()

,geometry,propertyid,node_id,grocery_2k,lightrail_2k,transit_cen_2k,food_drink_2k,cafe_2k,college_2k,entertainment_2k
0,POINT (-122.535763 45.781812),8021379,47251805,0.0,0.0,0.0,8.0,6.0,0.0,0.0
1,POINT (-122.5075064 45.6203498),9096130,47307668,0.0,0.0,0.0,5.0,4.0,0.0,0.0
2,POINT (-122.513618 45.622437),7769079,47330355,0.0,0.0,0.0,3.0,5.0,0.0,0.0
3,POINT (-122.669628 45.7208891),6058580,3686142098,0.0,0.0,0.0,5.0,5.0,0.0,0.0
4,POINT (-122.3796812 45.589126),5998841,47211778,0.0,0.0,0.0,16.0,4.0,0.0,1.0


In [24]:
# confirm join didn't exclude anyone
sub.shape

(409, 10)

In [25]:
# we can drop the geometry
sub.drop('geometry', axis=1, inplace=True)

### Parks

Parks require a different approach: being polygons they can't accurately be represented with one point, such as a centroid, for our purposes. How many park centroids an observation has within distance x is unecessarily restrictive. In practice what is more important is how many parks could be reached by traveling distance x? Put another way, how many park perimeters are within distance x? So here it seems traditional polygon to polygon intersection is the way to go. It is worth noting however that this will be a 'as the crow flies' measurement instead of a network traversal distance.

#### Read in the park layer

In [26]:
parks = gpd.read_file('./data/orca_sites')

In [27]:
parks.head(2)

,ACREAGE,DISSOLVEID,SITENAME,TYPE,geometry
0,2.627946,1.0,114th Avenue Wetlands Natural Area,Park and/or Natural Area,(POLYGON ((7614399.042322829 674357.0767716467...
1,1.008078,2.0,Sieben Park,Park and/or Natural Area,"POLYGON ((7684547.705380574 646138.3080708683,..."


In [28]:
parks = parks.rename(columns=lambda x: x.lower())

In [29]:
parks['type'].unique()

array(['Park and/or Natural Area', 'Cemetery', 'Golf Course',
       'Home Owner Association', 'School', 'Other'], dtype=object)

#### Filter down to just parks

In [30]:
parks = parks[parks['type'] == 'Park and/or Natural Area']

#### Check projections

In [31]:
props.crs

{'init': 'epsg:4326'}

In [32]:
parks.crs

{'ellps': 'GRS80',
 'lat_0': 43.66666666666666,
 'lat_1': 44.33333333333334,
 'lat_2': 46,
 'lon_0': -120.5,
 'no_defs': True,
 'proj': 'lcc',
 'towgs84': '0,0,0,0,0,0,0',
 'units': 'ft',
 'x_0': 2500000,
 'y_0': 0}

#### It generally seems best to use a local projection for these operations

In [33]:
props = props.to_crs({'init': 'epsg:2913'})

In [34]:
parks = parks.to_crs({'init': 'epsg:2913'})

#### Create a 2k buffer for each observation

In [35]:
# the units of this projection are feet: 6561.68 feet == 2k meters
props['geometry'] = props.apply(lambda row: row['geometry'].buffer(6561.68), axis=1)

In [36]:
props.head(2)

,geometry,propertyid,node_id
0,"POLYGON ((7689333.215643824 777684.8060858538,...",8021379,47251805
1,"POLYGON ((7695077.298893781 718648.9770450323,...",9096130,47307668


In [37]:
parks.head(2)

,acreage,dissolveid,sitename,type,geometry
0,2.627946,1.0,114th Avenue Wetlands Natural Area,Park and/or Natural Area,(POLYGON ((7614399.042790023 674357.0767716141...
1,1.008078,2.0,Sieben Park,Park and/or Natural Area,"POLYGON ((7684547.705847767 646138.3080708354,..."


### Do the spatial join

In [38]:
parks_sjoin = gpd.sjoin(props, parks[['sitename', 'geometry']], how='inner', op='intersects').drop('index_right', axis=1)

In [39]:
parks_sjoin.head(2)

,geometry,propertyid,node_id,sitename
6,"POLYGON ((7649923.877360179 680057.9612040246,...",6520531,40629129,Vera Katz Eastbank Esplanade
9,"POLYGON ((7655152.808667029 684534.6779698292,...",8766850,40682997,Vera Katz Eastbank Esplanade


In [40]:
parks_sjoin.shape

(9423, 4)

#### Group our observations to get back to the correct number of rows

In [41]:
parks_grp = parks_sjoin[['propertyid', 'sitename']].groupby(by='propertyid').count()

In [42]:
parks_grp.head(2)

,sitename
propertyid,
84126,28
673658,13


In [43]:
# not all properties are within 2k meters of a park so the number of rows here is less than 409
parks_grp.shape

(331, 1)

In [44]:
# rename the column
parks_grp.columns = ['parks_2k']

In [45]:
parks_grp.head(2)

,parks_2k
propertyid,
84126,28
673658,13


In [46]:
sub.head(2)

,propertyid,node_id,grocery_2k,lightrail_2k,transit_cen_2k,food_drink_2k,cafe_2k,college_2k,entertainment_2k
0,8021379,47251805,0.0,0.0,0.0,8.0,6.0,0.0,0.0
1,9096130,47307668,0.0,0.0,0.0,5.0,4.0,0.0,0.0


#### Merge parks_grp and sub to create the final dataframe

In [47]:
# the left join will leave some NaNs in the park column so they get filled as zeros
final = pd.merge(sub, parks_grp, how='left', left_on='propertyid', right_index=True).fillna(value=0)

In [48]:
final.head()

,propertyid,node_id,grocery_2k,lightrail_2k,transit_cen_2k,food_drink_2k,cafe_2k,college_2k,entertainment_2k,parks_2k
0,8021379,47251805,0.0,0.0,0.0,8.0,6.0,0.0,0.0,0.0
1,9096130,47307668,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0
2,7769079,47330355,0.0,0.0,0.0,3.0,5.0,0.0,0.0,0.0
3,6058580,3686142098,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0
4,5998841,47211778,0.0,0.0,0.0,16.0,4.0,0.0,1.0,0.0


### Add tract and blockgroup fips codes so observations can be joined to census data later
#### Read observations, tracts and blockgroups and confirm projections

In [49]:
props.head(2)

,geometry,propertyid,node_id
0,"POLYGON ((7689333.215643824 777684.8060858538,...",8021379,47251805
1,"POLYGON ((7695077.298893781 718648.9770450323,...",9096130,47307668


In [50]:
# reread the property file to get the original geometries
props = gpd.read_file('./data/properties')

In [51]:
props.crs

{'init': 'epsg:4326'}

In [52]:
props = props.to_crs({'init': 'epsg:2913'})

#### Tracts

In [53]:
# OR
tracts41 = gpd.read_file('./data/tracts_41')

In [54]:
tracts41.crs

{'init': 'epsg:4269'}

In [55]:
tracts41 = tracts41.to_crs({'init': 'epsg:2913'})

In [56]:
# WA
tracts53 = gpd.read_file('./data/tracts_53')

In [57]:
tracts53.crs

{'init': 'epsg:4269'}

In [58]:
tracts53 = tracts53.to_crs({'init': 'epsg:2913'})

In [59]:
tracts = pd.concat([tracts41, tracts53], ignore_index=True)

In [60]:
# pd operations do not always preserve gpd dataframes
type(tracts)

geopandas.geodataframe.GeoDataFrame

In [61]:
tracts.head(2)

,ALAND,AWATER,COUNTYFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,MTFCC,NAME,NAMELSAD,STATEFP,TRACTCE,geometry
0,8828148,4127662,015,S,41015950302,+42.0617567,-124.2939759,G5020,9503.02,Census Tract 9503.02,41,950302,"POLYGON ((7161539.169300538 -558639.417910337,..."
1,621349742,4962481,015,S,41015950400,+42.1168650,-123.9370837,G5020,9504,Census Tract 9504,41,950400,POLYGON ((7173981.973703014 -569930.5500462982...


In [62]:
tracts = tracts.rename(columns=lambda x: x.lower())

#### Block groups

In [63]:
# OR
bgs41 = gpd.read_file('./data/bgs_41')

In [64]:
bgs41.crs

{'init': 'epsg:4269'}

In [65]:
bgs41 = bgs41.to_crs({'init': 'epsg:2913'})

In [66]:
# WA
bgs53 = gpd.read_file('./data/bgs_53')

In [67]:
bgs53.crs

{'init': 'epsg:4269'}

In [68]:
bgs53 = bgs53.to_crs({'init': 'epsg:2913'})

In [69]:
bgs = pd.concat([bgs41, bgs53], ignore_index=True)

In [70]:
type(bgs)

geopandas.geodataframe.GeoDataFrame

In [71]:
bgs = bgs.rename(columns=lambda x: x.lower())

### Perform spatial joins to assign each observation to a geography

In [72]:
tracts.head(2)

,aland,awater,countyfp,funcstat,geoid,intptlat,intptlon,mtfcc,name,namelsad,statefp,tractce,geometry
0,8828148,4127662,015,S,41015950302,+42.0617567,-124.2939759,G5020,9503.02,Census Tract 9503.02,41,950302,"POLYGON ((7161539.169300538 -558639.417910337,..."
1,621349742,4962481,015,S,41015950400,+42.1168650,-123.9370837,G5020,9504,Census Tract 9504,41,950400,POLYGON ((7173981.973703014 -569930.5500462982...


In [73]:
bgs.head(2)

,aland,awater,blkgrpce,countyfp,funcstat,geoid,intptlat,intptlon,mtfcc,namelsad,statefp,tractce,geometry
0,389639,0,4,051,S,410510012014,+45.5171764,-122.6443508,G5030,Block Group 4,41,001201,"POLYGON ((7651156.260178134 681481.2395682092,..."
1,193207,0,1,051,S,410510013021,+45.5104150,-122.6261822,G5030,Block Group 1,41,001302,"POLYGON ((7656226.54015459 679053.8458619422, ..."


In [74]:
census_geos = gpd.sjoin(props, tracts[['geoid', 'geometry']], how='inner', op='within')

In [75]:
census_geos.shape

(409, 4)

In [76]:
census_geos.head(2)

,geometry,propertyid,index_right,geoid
0,POINT (7682771.535643824 777684.8060858538),8021379,1910,53011040415
1,POINT (7688515.618893782 718648.9770450323),9096130,1964,53011041320


In [77]:
census_geos.drop('index_right', axis=1, inplace=True)

In [78]:
census_geos = census_geos.rename(columns={'geoid': 'tract_fips'})

In [79]:
census_geos.head(2)

,geometry,propertyid,tract_fips
0,POINT (7682771.535643824 777684.8060858538),8021379,53011040415
1,POINT (7688515.618893782 718648.9770450323),9096130,53011041320


In [80]:
census_geos = gpd.sjoin(census_geos, bgs[['geoid', 'geometry']], how='inner', op='within')

In [81]:
census_geos.drop('index_right', axis=1, inplace=True)

In [82]:
census_geos = census_geos.rename(columns={'geoid': 'bg_fips'})

In [83]:
census_geos.head(2)

,geometry,propertyid,tract_fips,bg_fips
0,POINT (7682771.535643824 777684.8060858538),8021379,53011040415,530110404151
1,POINT (7688515.618893782 718648.9770450323),9096130,53011041320,530110413204


### One final merge of all the data

In [84]:
master = pd.merge(final, census_geos, how='inner', on='propertyid')

In [85]:
master.shape

(409, 13)

In [86]:
master.head(2)

,propertyid,node_id,grocery_2k,lightrail_2k,transit_cen_2k,food_drink_2k,cafe_2k,college_2k,entertainment_2k,parks_2k,geometry,tract_fips,bg_fips
0,8021379,47251805,0.0,0.0,0.0,8.0,6.0,0.0,0.0,0.0,POINT (7682771.535643824 777684.8060858538),53011040415,530110404151
1,9096130,47307668,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0.0,POINT (7688515.618893782 718648.9770450323),53011041320,530110413204


In [87]:
# geometry is no longer needed
master.drop('geometry', axis=1, inplace=True)

In [88]:
master.to_csv('./output.csv', index=False)

### This data could now be joined on 'propertyid' back to the rent data and these spatial variables could be used as regressors such as 

\begin{equation*}  rent = \beta_0 + \beta_1(grocery_2k) + \beta_2(lightrail_2k) +  ...  + \epsilon \end{equation*}